# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located in the lab directory named `Sentiment140.csv.zip`. You need to unzip it into a `.csv` file. Then in the cell below, load and explore the data.

*Notes:* 

* The dataset was downloaded from [Kaggle](https://www.kaggle.com/kazanova/sentiment140). We made a slight change on the original data so that each column has a label.

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [3]:
import pandas as pd
import re


In [4]:
# your code here
df = pd.read_csv('Sentiment140.csv')
df.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [5]:
df2 = df.sample(n = 20000)


In [6]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 133331 to 317522
Data columns (total 6 columns):
target    20000 non-null int64
ids       20000 non-null int64
date      20000 non-null object
flag      20000 non-null object
user      20000 non-null object
text      20000 non-null object
dtypes: int64(2), object(4)
memory usage: 1.1+ MB


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [7]:
# your code here
#function 1
def clean_up(s):
    
    s = re.sub(r'[^\w\s]',' ',s)
    s = s.replace("http", "...")
    s = re.sub(r'\.\.\..*',"",s)
    s = re.sub(r'\d+', '', s)
    newstring = s.lower()
    return newstring
#function 2
import nltk

def tokenize(s):
    
    s = s.split()
    return s

#function 3
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download("wordnet")

def stem_and_lemmatize(l):
    lemmatizer = WordNetLemmatizer()
    lst =[]
    for x in l:
        word = SnowballStemmer("english").stem(x)
        word_lem = lemmatizer.lemmatize(word)
        lst.append(word_lem)
            
    return lst
#function 3
from nltk.corpus import stopwords 
nltk.download('stopwords')

def remove_stopwords(l):
    stop_words = set(stopwords.words('english'))
    filtered_word = [] 
    for x in l: 
        if x not in stop_words: 
            filtered_word.append(x)
            
    return filtered_word    

df2["text_processed"] = df2["text"].apply(clean_up).apply(tokenize).apply(stem_and_lemmatize).apply(remove_stopwords)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/alessandro/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alessandro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
df2.head()

,target,ids,date,flag,user,text,text_processed
133331,0,1835975266,Mon May 18 07:07:41 PDT 2009,NO_QUERY,clomacdee,stupid insurance companies... hurry up i need ...,"[stupid, insur, compani, hurri, need, new, ipod]"
818126,4,1552002526,Sat Apr 18 10:31:55 PDT 2009,NO_QUERY,Lovablexox,Twittering . And looking for Miley Cyrus LUv ...,"[twitter, look, miley, cyrus, luv, xo]"
1133544,4,1976196466,Sat May 30 16:47:53 PDT 2009,NO_QUERY,therealphoenix,spreading my love of jb by having people watch...,"[spread, love, jb, peopl, watch, paranoid, mv,..."
1090010,4,1969757151,Sat May 30 01:06:49 PDT 2009,NO_QUERY,flawedangel,@Hadramie Thanks! I am trying that right this ...,"[hadrami, thank, tri, right, second, hope, men..."
1152828,4,1978757836,Sun May 31 00:22:01 PDT 2009,NO_QUERY,joshwells,"@NIKKISALYER that sounds a lot like me, its no...","[nikkisaly, sound, lot, like, weird]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [9]:
# your code here

import pandas as pd
from nltk.probability import FreqDist

#data = pd.read_csv("X:\Firm_Data.csv")
#top_N = 20

word = []
for lst in list(df2['text_processed']):
    word.append(lst)


word_final = [inner for outer in word for inner in outer]

#dic = dict(FreqDist(word_final).most_common(5000))
#dic

all_words = nltk.FreqDist(word_final)
word_features = list(all_words.keys())[:5000]
word_features

['stupid',
 'insur',
 'compani',
 'hurri',
 'need',
 'new',
 'ipod',
 'twitter',
 'look',
 'miley',
 'cyrus',
 'luv',
 'xo',
 'spread',
 'love',
 'jb',
 'peopl',
 'watch',
 'paranoid',
 'mv',
 'wear',
 'tee',
 'sing',
 'song',
 'hadrami',
 'thank',
 'tri',
 'right',
 'second',
 'hope',
 'menjadik',
 'nikkisaly',
 'sound',
 'lot',
 'like',
 'weird',
 'mattybeauti',
 'mike',
 'honcho',
 'one',
 'best',
 'denver',
 'dubstep',
 'dj',
 'pas',
 'away',
 'rememb',
 'see',
 'show',
 'year',
 'ago',
 'dbrownstein',
 'care',
 'trip',
 'nd',
 'relax',
 'eat',
 'nummi',
 'browni',
 'amp',
 'wait',
 'blue',
 'black',
 'berri',
 'enzym',
 'facial',
 'mask',
 'work',
 'deeeeep',
 'breeeath',
 'cat',
 'coolmomofj',
 'cleavleand',
 'lost',
 'susan',
 'boyl',
 'okaaaay',
 'irldext',
 'photo',
 'dan',
 'zentechguy',
 'taken',
 'green',
 'gulch',
 'glad',
 'saw',
 '_',
 'amous',
 'awwwww',
 'sorri',
 'hear',
 'get',
 'hour',
 'min',
 'danger_ski',
 'girl',
 'know',
 'ha',
 'bub',
 'wa',
 'month',
 'premmi

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [35]:
# your code here

def find_features(document):
    words = set(document)
    features = {} 
    for w in word_features:
        features[w]=(w in words)
    return features

features = find_features(word_final)
features

{'stupid': True,
 'insur': True,
 'compani': True,
 'hurri': True,
 'need': True,
 'new': True,
 'ipod': True,
 'twitter': True,
 'look': True,
 'miley': True,
 'cyrus': True,
 'luv': True,
 'xo': True,
 'spread': True,
 'love': True,
 'jb': True,
 'peopl': True,
 'watch': True,
 'paranoid': True,
 'mv': True,
 'wear': True,
 'tee': True,
 'sing': True,
 'song': True,
 'hadrami': True,
 'thank': True,
 'tri': True,
 'right': True,
 'second': True,
 'hope': True,
 'menjadik': True,
 'nikkisaly': True,
 'sound': True,
 'lot': True,
 'like': True,
 'weird': True,
 'mattybeauti': True,
 'mike': True,
 'honcho': True,
 'one': True,
 'best': True,
 'denver': True,
 'dubstep': True,
 'dj': True,
 'pas': True,
 'away': True,
 'rememb': True,
 'see': True,
 'show': True,
 'year': True,
 'ago': True,
 'dbrownstein': True,
 'care': True,
 'trip': True,
 'nd': True,
 'relax': True,
 'eat': True,
 'nummi': True,
 'browni': True,
 'amp': True,
 'wait': True,
 'blue': True,
 'black': True,
 'berri': 

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [32]:
# your code here
import random


training_set = features[:16000]
testing_set = features[16000:]

TypeError: unhashable type: 'slice'

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [33]:
# your code here
classifier = nltk.NaiveBayesClassifier.train(training_set)
nltk.classify.accuracy(classifier, testing_set)

ValueError: A ELE probability distribution must have at least one bin.

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here